In [1]:
from dolfinx import mesh, fem, log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import ufl
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc

domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)
element = ufl.FiniteElement("CG", domain.ufl_cell(), 2)
V = fem.FunctionSpace(domain, element)

u = fem.Function(V)
v = ufl.TestFunction(V)

x = ufl.SpatialCoordinate(domain)
Fx = (u.dx(0) - 2 * x[0]) * v * ufl.dx
Fy = (u.dx(1) - 2 * x[1]) * v * ufl.dx
F = Fx + Fy

In [2]:
bc = fem.dirichletbc(default_scalar_type(0), np.array([0], dtype=np.int32), V)
problem = NonlinearProblem(F, u, [bc])

In [3]:
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "residual"
solver.rtol = 1e-5
solver.atol = 1e-5
solver.max_it = 50
solver.relaxation_parameter = 1
# We can customize the linear solver used inside the NewtonSolver by
# modifying the PETSc options
ksp = solver.krylov_solver
opts = PETSc.Options()  # type: ignore
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
ksp.setFromOptions()
log.set_log_level(log.LogLevel.INFO)
solver.solve(u)

2023-12-17 16:47:37.458 (   0.152s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 0: r (abs) = 0.122273 (tol = 1e-05) r (rel) = inf(tol = 1e-05)
2023-12-17 16:47:37.459 (   0.153s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2023-12-17 16:47:37.462 (   0.156s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 1: r (abs) = 4.48574e-14 (tol = 1e-05) r (rel) = 1.49283e-18(tol = 1e-05)
2023-12-17 16:47:37.462 (   0.156s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 1 iterations and 1 linear solver iterations.


(1, True)

In [4]:
import utils

utils.plot_function(u, V)
u.x.array

Widget(value='<iframe src="http://localhost:40835/index.html?ui=P_0x7f38cbb33110_0&reconnect=auto" class="pyvi…

array([ 0.00000000e+00,  2.72598491e+04,  2.00000000e-01, -8.78512335e+03,
        9.50000000e-02, -8.78522335e+03,  3.05957858e+02,  3.44182176e+02,
        3.44082176e+02,  3.05777858e+02,  3.05862858e+02,  3.43992176e+02,
        3.06177858e+02,  3.44292176e+02,  3.06062858e+02, -2.69254324e+02,
        1.05359639e+02,  1.05269639e+02, -2.69054324e+02,  1.05569639e+02,
        1.05459639e+02, -2.69414324e+02, -2.69339324e+02,  1.05189639e+02,
       -2.69159324e+02, -2.68814324e+02,  1.05689639e+02, -2.68939324e+02,
       -6.92304845e+01,  9.97421338e+00,  9.89421338e+00, -6.90504845e+01,
        1.01642134e+01,  1.00642134e+01, -6.88304845e+01,  1.03942134e+01,
        1.02742134e+01, -6.93704845e+01, -6.93054845e+01,  9.82421338e+00,
       -6.91454845e+01, -6.89454845e+01, -6.85704845e+01,  1.05242134e+01,
       -6.87054845e+01, -1.57608930e+01, -8.21125950e+01, -8.21825950e+01,
       -1.56008930e+01, -8.19425950e+01, -8.20325950e+01, -1.54008930e+01,
       -8.17325950e+01, -